# Capstone Project - The Battle of the Neighborhoods (Week 2)




## Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to recommend restaurant to XYZ Company.

Since there are lots of restaurants in Berlin we will try to detect **locations of restaurants and recommend**. We are also particularly interested in **areas with restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by XYZ Company.

## Data 

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**


In [2]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Alexanderplatz, Berlin, Germany'
berlin_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, berlin_center))

Coordinate of Alexanderplatz, Berlin, Germany: [52.5219184, 13.4132147]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Alexanderplatz. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [5]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Berlin center longitude={}, latitude={}'.format(berlin_center[1], berlin_center[0]))
x, y = lonlat_to_xy(berlin_center[1], berlin_center[0])
print('Berlin center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Berlin center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Berlin center longitude=13.4132147, latitude=52.5219184
Berlin center UTM X=392341.28017572395, Y=5820273.243274779
Berlin center longitude=13.413214700000001, latitude=52.52191839999997


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [6]:
berlin_center_x, berlin_center_y = lonlat_to_xy(berlin_center[1], berlin_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = berlin_center_x - 6000
x_step = 600
y_min = berlin_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(berlin_center_x, berlin_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Alexanderplatz. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [12]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, berlin_center[0], berlin_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(berlin_center[0], berlin_center[1], addr))

Reverse geocoding check
-----------------------
Address of [52.5219184, 13.4132147] is: Alexanderpl. 5, 10178 Berlin, Germany


In [13]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Germany', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [14]:
addresses[150:170]

['Frankfurter Allee 147-149, 10365 Berlin',
 'Magdalenenstraße 12, 10365 Berlin',
 'Siegfriedstraße 207, 10365 Berlin',
 'Englische Str. 3, 10587 Berlin',
 'Händelallee 51, 10557 Berlin',
 'Spreeweg, 10557 Berlin',
 'John-Foster-Dulles-Allee 10, 10557 Berlin',
 'B96, 10557 Berlin',
 'Pariser Platz 6A, 10117 Berlin',
 'Unter den Linden 38, 10117 Berlin',
 'Unter den Linden 5, 10117 Berlin',
 'Spreeufer 6, 10178 Berlin',
 'Parochialstraße, 10179 Berlin',
 'Neue Blumenstraße 1, 10179 Berlin',
 'Blumenstraße 41, 10243 Berlin',
 'B5 85, 10243 Berlin',
 'Weidenweg 27, 10249 Berlin',
 'Rigaer Str. 96, 10247 Berlin',
 'Bänschstraße 58, 10247 Berlin',
 'Parkaue 30, 10367 Berlin']

Looking good. Let's now place all this into a Pandas dataframe.

In [15]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Distance from center,Latitude,Longitude,X,Y
0,"Bundesautobahn 100 & Tempelhofer Damm, 12099 B...",5992.495307,52.470194,13.388575,390541.280176,5.814557e+06
1,"09R/27L, 12101 Berlin",5840.376700,52.470314,13.397404,391141.280176,5.814557e+06
2,"09R/27L, 12049 Berlin",5747.173218,52.470434,13.406234,391741.280176,5.814557e+06
3,"09R/27L, 12049 Berlin",5715.767665,52.470552,13.415063,392341.280176,5.814557e+06
4,"Warthestraße 23, 12051 Berlin",5747.173218,52.470670,13.423893,392941.280176,5.814557e+06
5,"Schierker Str. 19-20, 12051 Berlin",5840.376700,52.470788,13.432722,393541.280176,5.814557e+06
6,"Karl-Marx-Straße 213, 12055 Berlin",5992.495307,52.470904,13.441552,394141.280176,5.814557e+06
7,"Hessenring 34, 12101 Berlin",5855.766389,52.474683,13.375159,389641.280176,5.815077e+06
8,"Thuyring 6, 12101 Berlin",5604.462508,52.474804,13.383989,390241.280176,5.815077e+06
9,"09L/27R, 12101 Berlin",5408.326913,52.474924,13.392820,390841.280176,5.815077e+06


...and let's now save/persist this data into local file.

In [16]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on restaurants in the neighborhood.

In [18]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [19]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [20]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 2031
Total number of Italian restaurants: 312
Percentage of Italian restaurants: 15.36%
Average number of restaurants in neighborhood: 4.91208791209


In [21]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('5546072a498e349bf0e737e1', 'Shaam Restaurant', 52.474363806181806, 13.440120220184326, 'Karl-Marx-Straße 177, 10247 Berlin', 249, False, 394052.35775333317, 5814944.355430137)
('4fce25c6e4b0f39fffdd0447', 'Wursterei', 52.5058278495275, 13.333072532529153, 'Hardenbergplatz 27d, 10623 Berlin', 133, False, 386862.9315917266, 5818606.191572046)
('57ffdde438fa512462a6b490', 'Einstein Kaffeehaus & Restaurant', 52.516953, 13.385849, 'Unter der Linden 42, 10117 Berlin', 69, False, 390472.37417370133, 5819762.151308152)
('514316eae4b080a105a5b4f5', 'Allee Bistro', 52.534855836549994, 13.497241138618675, 'Berlin', 279, False, 398071.8391866421, 5821590.182515125)
('4c3a05951a38ef3b86079321', 'Louis', 52.474274260971214, 13.445097179795765, 'Richardplatz 5, 12055 Berlin', 158, False, 394390.1589999274, 5814927.10762019)
('4b62bc3df964a520b4502ae3', 'Kaplan Döner', 52.556723244788124, 13.373655087007442, 'Osloer Str. 84, Berlin', 248, False, 389744

In [22]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('4b4f6063f964a520e10327e3', 'Salumeria Culinario', 52.526394678482745, 13.393537136029817, 'Tucholskystr. 34 (Auguststr.), 10117 Berlin', 123, True, 391017.3861974631, 5820800.631260842)
('56d5838e498eda2c7124a8f0', 'Pascarella', 52.53224028238963, 13.380982905663293, 'Berlin', 168, True, 390180.3491140888, 5821469.816676741)
('4f1ff655e4b0ec749c54b273', 'Agata Torrisi', 52.53651019364004, 13.377780874234496, 'Wöhlertstr. 5, 10115 Berlin', 121, True, 389973.84217276424, 5821949.599236318)
('551ecd4e498e52f76b5f4310', "Antonello's Cevicheria & Street Food", 52.49042751718539, 13.390365472982154, 'Nostitzstr. 22, Berlin', 240, True, 390713.04239021626, 5816805.116119504)
('4bf2dd126991c9b6629829e9', 'Al Contadino Sotto Le Stelle', 52.52780835209213, 13.401225263437848, 'Auguststr. 36 (Joachimstr.), 10115 Berlin', 250, True, 391542.39733332, 5820946.282045525)
('4afc5179f964a5207e2122e3', 'Boccondivino', 52.522249160938536, 13.38431

In [23]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Mabuhay, Scandic Restaurant
Restaurants around location 102: Solar, THE POST Brasserie & Bar, Ristorante Marinelli, Diomira, Mexican, Cucina Italiana, Restaurant Hof zwei, Morélos Steakhaus & Cocktailbar
Restaurants around location 103: Paracas, Nobelhart & Schmutzig, Mama Cook, Trattoria da Vinci, Steakhaus Asador, Tumi, Delhi 6, Deutsche Küche By Kaese-koenig.de
Restaurants around location 104: 
Restaurants around location 105: Pacifico, food bag 2, TAT Imbiss
Restaurants around location 106: Santa Maria, Die Henne, Zur kleinen Markthalle, Parantez, Habibi, Maroush, Sol y Sombra, Chez Michel
Restaurants around location 107: La Piadina, 3 Schwestern, Trattoria Marechiaro, Goldener Hahn, Weltrestaurant Markthalle, Long March Canteen, Olive
Restaurants around location 108: Salumeria Lamuri, Restaurant Richard
Restaurants around location 109: Scheers Schnitzel, Seoulkitchen Korean BBQ & Sushi, Michel

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Berlin that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from Berlin center** (Alexanderplatz). We have also **identified Italian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Berlin - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Italian restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [31]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 4.91208791209


,Address,Distance from center,Latitude,Longitude,X,Y,Restaurants in area
0,"Bundesautobahn 100 & Tempelhofer Damm, 12099 B...",5992.495307,52.470194,13.388575,390541.280176,5.814557e+06,4
1,"09R/27L, 12101 Berlin",5840.376700,52.470314,13.397404,391141.280176,5.814557e+06,0
2,"09R/27L, 12049 Berlin",5747.173218,52.470434,13.406234,391741.280176,5.814557e+06,0
3,"09R/27L, 12049 Berlin",5715.767665,52.470552,13.415063,392341.280176,5.814557e+06,0
4,"Warthestraße 23, 12051 Berlin",5747.173218,52.470670,13.423893,392941.280176,5.814557e+06,1
5,"Schierker Str. 19-20, 12051 Berlin",5840.376700,52.470788,13.432722,393541.280176,5.814557e+06,6
6,"Karl-Marx-Straße 213, 12055 Berlin",5992.495307,52.470904,13.441552,394141.280176,5.814557e+06,5
7,"Hessenring 34, 12101 Berlin",5855.766389,52.474683,13.375159,389641.280176,5.815077e+06,0
8,"Thuyring 6, 12101 Berlin",5604.462508,52.474804,13.383989,390241.280176,5.815077e+06,0
9,"09L/27R, 12101 Berlin",5408.326913,52.474924,13.392820,390841.280176,5.815077e+06,0


OK, now let's calculate the **distance to nearest Italian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [32]:
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant

In [33]:
df_locations.head(10)

,Address,Distance from center,Latitude,Longitude,X,Y,Restaurants in area,Distance to Italian restaurant
0,"Bundesautobahn 100 & Tempelhofer Damm, 12099 B...",5992.495307,52.470194,13.388575,390541.280176,5.814557e+06,4,264.408532
1,"09R/27L, 12101 Berlin",5840.376700,52.470314,13.397404,391141.280176,5.814557e+06,0,830.999331
2,"09R/27L, 12049 Berlin",5747.173218,52.470434,13.406234,391741.280176,5.814557e+06,0,1269.038823
3,"09R/27L, 12049 Berlin",5715.767665,52.470552,13.415063,392341.280176,5.814557e+06,0,829.067436
4,"Warthestraße 23, 12051 Berlin",5747.173218,52.470670,13.423893,392941.280176,5.814557e+06,1,575.681166
5,"Schierker Str. 19-20, 12051 Berlin",5840.376700,52.470788,13.432722,393541.280176,5.814557e+06,6,293.966217
6,"Karl-Marx-Straße 213, 12055 Berlin",5992.495307,52.470904,13.441552,394141.280176,5.814557e+06,5,317.390305
7,"Hessenring 34, 12101 Berlin",5855.766389,52.474683,13.375159,389641.280176,5.815077e+06,0,776.047531
8,"Thuyring 6, 12101 Berlin",5604.462508,52.474804,13.383989,390241.280176,5.815077e+06,0,378.018237
9,"09L/27R, 12101 Berlin",5408.326913,52.474924,13.392820,390841.280176,5.815077e+06,0,635.252552


In [35]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 495.2099580523902


OK, so **on average Italian restaurant can be found within ~500m** from every area center candidate. That's fairly close, so we need to filter our areas carefully!

Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show **borders of Berlin boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km from Alexanderplatz.

In [36]:
berlin_boroughs_url = 'https://raw.githubusercontent.com/m-hoerz/berlin-shapes/master/berliner-bezirke.geojson'
berlin_boroughs = requests.get(berlin_boroughs_url).json()

def boroughs_style(feature):
    return { 'color': 'blue', 'fill': False }

In [37]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]

Looks like a few pockets of low restaurant density closest to city center can be found **south, south-east and east from Alexanderplatz**. 

Let's create another heatmap map showing **heatmap/density of Italian restaurants** only.

This map is not so 'hot' (Italian restaurants represent a subset of ~15% of all restaurants in Berlin) but it also indicates higher density of existing Italian restaurants directly north and west from Alexanderplatz, with closest pockets of **low Italian restaurant density positioned east, south-east and south from city center**.

Based on this we will now focus our analysis on areas *south-west, south, south-east and east from Berlin center* - we will move the center of our area of interest and reduce it's size to have a radius of **2.5km**. This places our location candidates mostly in boroughs **Kreuzberg and Friedrichshain** (another potentially interesting borough is **Prenzlauer Berg** with large low restaurant density north-east from city center, however this borough is less interesting to stakeholders as it's mostly residental and less popular with tourists).

In [41]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

2261 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Italian restaurant**.

In [94]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_italian_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, italian_restaurants)
    roi_italian_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [95]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Italian restaurant':roi_italian_distances})

df_roi_locations.head(10)

,Distance to Italian restaurant,Latitude,Longitude,Restaurants nearby,X,Y
0,158.565914,52.486060,13.421133,8,392791.280176,5.816273e+06
1,180.522171,52.486080,13.422605,9,392891.280176,5.816273e+06
2,516.071190,52.486730,13.413009,0,392241.280176,5.816360e+06
3,468.257436,52.486750,13.414481,0,392341.280176,5.816360e+06
4,369.743331,52.486769,13.415953,0,392441.280176,5.816360e+06
5,272.314591,52.486789,13.417425,3,392541.280176,5.816360e+06
6,177.764848,52.486809,13.418897,6,392641.280176,5.816360e+06
7,95.107551,52.486829,13.420369,8,392741.280176,5.816360e+06
8,80.563958,52.486848,13.421841,9,392841.280176,5.816360e+06
9,154.711526,52.486868,13.423314,12,392941.280176,5.816360e+06


OK. Let us now **filter** those locations: we're interested only in **locations with no more than two restaurants in radius of 250 meters**, and **no Italian restaurants in radius of 400 meters**.

In [98]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Italian restaurant']>=400)
print('Locations with no Italian restaurants within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 798
Locations with no Italian restaurants within 400m: 380
Locations with both conditions met: 319


In [86]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', Germany', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, berlin_center_x, berlin_center_y)
    print('{}{} => {:.1f}km from Alexanderplatz'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

Michaelkirchpl. 15, 10179 Berlin                   => 1.7km from Alexanderplatz
Lohmühlenstraße 1, 12435 Berlin                    => 3.7km from Alexanderplatz
Berolinastraße 12, 10178 Berlin                    => 0.5km from Alexanderplatz
Neuenburger Str. 15, 10969 Berlin                  => 2.7km from Alexanderplatz
Landsberger Allee 37, 10249 Berlin                 => 2.0km from Alexanderplatz
Bona-Peiser-Weg 4, 10179 Berlin                    => 1.7km from Alexanderplatz
Gitschiner Str. 33, 10969 Berlin                   => 2.7km from Alexanderplatz
Stallschreiberstraße 48, 10969 Berlin              => 1.8km from Alexanderplatz
An der Ostbahn 5, 10243 Berlin                     => 2.5km from Alexanderplatz
Hasenheide 81, 10967 Berlin                        => 3.9km from Alexanderplatz
Ifflandstraße 9, 10179 Berlin                      => 0.9km from Alexanderplatz
Reichenberger Str. 92, 10999 Berlin                => 3.

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants and no Italian restaurants nearby, all zones being fairly close to city center (all less than 4km from Alexanderplazt, and about half of those less than 2km from Alexanderplatz). Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Kreuzberg and Friedrichshain boroughs, which we have identified as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Berlin (~2000 in our initial area of interest which was 12x12km around Alexanderplatz), there are pockets of low restaurant density fairly close to city center. Highest concentration of restaurants was detected north and west from Alexanderplatz, so we focused our attention to areas south, south-east and east, corresponding to boroughs Kreuzberg, Friedrichshain and south-east corner of central Mitte borough. Another borough was identified as potentially interesting (Prenzlauer Berg, north-east from Alexanderplatz), but our attention was focused on Kreuzberg and Friedrichshain which offer a combination of popularity among tourists, closeness to city center, strong socio-economic dynamics *and* a number of pockets of low restaurant density.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Berlin areas close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by XYZ Company.